Set up

In [ ]:
# Install environment and agent
!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
!pip install git+https://github.com/DLR-RM/stable-baselines3

# Environment
import gymnasium as gym
import highway_env

# Agent
from stable_baselines3 import DQN

# Visualization utils
%load_ext tensorboard
import sys
from tqdm.notebook import trange
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb ffmpeg
!git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
#from utils import record_videos, show_videos

  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-zesq65lo
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-zesq65lo
  Resolved https://github.com/DLR-RM/stable-baselines3 to commit f375cc393938a6b4e4dc0fb1de82b4afca37c1bd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


Testing

In [27]:
!apt-get install -y xvfb ffmpeg
import sys
import copy
import numpy as np
import warnings
from gymnasium.wrappers import RecordVideo


sys.path.insert(0, '/content/HighwayEnv/scripts/')
warnings.filterwarnings("ignore", category=DeprecationWarning)

env1 = gym.make('highway-fast-v0', render_mode='rgb_array')
env1.configure({
    "action": {
        "type": "DiscreteMetaAction",
    },
    "lanes_count": 4,
    "vehicles_count": 50,
    "duration": 100,  # [s]
    "initial_spacing": 2,
    "collision_reward": -10,  # The reward received when colliding with a vehicle.
    "reward_speed_range": [20, 80],  # [m/s] The reward for high speed is mapped linearly from this range to [0, HighwayEnv.HIGH_SPEED_REWARD].
    "simulation_frequency": 15,  # [Hz]
    "policy_frequency": 1,  # [Hz]
    "high_speed_reward": 5,
    "right_lane_reward": 0,
    "lane_change_reward": 0,
    "other_vehicles_type": "highway_env.vehicle.behavior.IDMVehicle",
    "screen_width": 600,  # [px]
    "screen_height": 150,  # [px]
    "centering_position": [0.3, 0.5],
    "scaling": 5.5,
    "show_trajectories": False,
    "render_agent": True,
    "offscreen_rendering": False
})

env2 = copy.deepcopy(env1)
env3 = copy.deepcopy(env1)


model1 = DQN.load("dqn_fastright.zip", env1, custom_objects={'observation_space': env1.observation_space, 'action_space': env1.action_space})
model2 = DQN.load("dqn_social_wreward.zip", env2, custom_objects={'observation_space': env2.observation_space, 'action_space': env2.action_space})
model3 = DQN.load("dqn_parallelrew.zip", env3, custom_objects={'observation_space': env3.observation_space, 'action_space': env3.action_space})

env1 = RecordVideo(env1, video_folder="run1",
              episode_trigger=lambda e: True)  # record all episodes
# Provide the video recorder to the wrapped environment
# so it can send it intermediate simulation frames.
env1.unwrapped.set_record_video_wrapper(env1)

env2 = RecordVideo(env2, video_folder="run2",
              episode_trigger=lambda e: True)  # record all episodes
# Provide the video recorder to the wrapped environment
# so it can send it intermediate simulation frames.
env2.unwrapped.set_record_video_wrapper(env2)
#env2 = record_videos(env2)

env3 = RecordVideo(env3, video_folder="run3",
              episode_trigger=lambda e: True)  # record all episodes
# Provide the video recorder to the wrapped environment
# so it can send it intermediate simulation frames.
env3.unwrapped.set_record_video_wrapper(env3)

#for episode in trange(1, desc='Test episodes'):
(obs1, _) = env1.reset()
(obs2, _) = env2.reset()
(obs3, _) = env3.reset()

done1, done2, done3 = False, False, False
assert obs1.tolist() == obs2.tolist(), f'Nonidentical environment'
assert obs1.tolist() == obs3.tolist(), f'Nonidentical environment'

while not (done1 or done2 or done3):
    action1, _ = model1.predict(obs1, deterministic=True)
    obs1, reward1, done1, truncated1, info1 = env1.step(int(action1))

    action2, _ = model2.predict(obs2, deterministic=True)
    obs2, reward2, done2, truncated2, info2 = env2.step(int(action2))

    action3, _ = model3.predict(obs3, deterministic=True)
    obs3, reward3, done3, truncated3, info3 = env3.step(int(action3))

    env1.render()
    env2.render()
    env3.render()

env1.close()
env2.close()
env3.close()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


  logger.warn(

  logger.warn(

  logger.warn(

  logger.warn(



Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Moviepy - Building video /content/run1/rl-video-episode-0.mp4.
Moviepy - Writing video /content/run1/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/run1/rl-video-episode-0.mp4
Moviepy - Building video /content/run2/rl-video-episode-0.mp4.
Moviepy - Writing video /content/run2/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/run2/rl-video-episode-0.mp4
Moviepy - Building video /content/run3/rl-video-episode-0.mp4.
Moviepy - Writing video /content/run3/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/run3/rl-video-episode-0.mp4
